# Demo of spectral extractions and analysis as of May 2024

Note this is a copy of Gabe Brammer's msaexp example notebook: https://github.com/gbrammer/msaexp/tree/main/docs/examples

We will use this notebook to do some high level work on reduced data such as fitting redshifts and obtaining line fluxes etc. 


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import yaml

import astropy.units as u
import astropy.io.fits as pyfits

import mastquery.utils
from grizli import utils
import msaexp.slit_combine
import msaexp.spectrum
import msaexp

print('msaexp version: ', msaexp.__version__)

## A reduced RUBIES spectrum

https://s3.amazonaws.com/msaexp-nirspec/extractions/rubies-egs61-v2/index.html

In [ ]:
spec_file = 'rubies-egs61-v2_prism-clear_4233_44597.spec.fits'

srcid = spec_file.split('_')[-1].split('.spec.fits')[0]
spec_root = spec_file.split('_')[0]
spec_url = f'https://s3.amazonaws.com/msaexp-nirspec/extractions/{spec_root}/{spec_file}'

spec_hdu = pyfits.open(spec_url)
spec_hdu.writeto(spec_file, overwrite=True)

spec_hdu.info()

In [ ]:
spec = utils.read_catalog(spec_file)
spec.info()

In [ ]:
# Lots of metadata
print(yaml.dump(dict(spec.meta)))

### Shutter footprints

See if you can match the image you receive here with the APT using ALADIN

In [ ]:
import PIL
import urllib

# Cutout figure
cutout_url = "https://grizli-cutout.herokuapp.com/thumb?size=1&scl=4.0&invert=True&filters=f444w-clear&rgb_scl=1.5%2C0.74%2C1.3&pl=2&coord={SRCRA},{SRCDEC}&nirspec=True&dpi_scale=6&nrs_source=magenta&nrs_other=magenta&nrs_lw=0.5&nrs_alpha=0.8"

img = np.array(PIL.Image.open(urllib.request.urlopen(cutout_url.format(**spec.meta))))

fig, ax = plt.subplots(1,1,figsize=(5,5))
_ = ax.imshow(img[::-1,:,:])
ax.axis('off')

In [ ]:
# The actual footprints
slits_url = "https://grizli-cutout.herokuapp.com/nirspec_slits?coord={SRCRA},{SRCDEC}"
slits = utils.read_catalog(slits_url.format(**spec.meta), format='csv')
slits['program','source_id','grating','is_source','footprint']

### Read the MAST spectrum for comparison

In [ ]:
mast_file = 'jw{PROGRAM}-o{OBSERVTN}_s{SOURCEID}_nirspec_{FILTER}-{GRATING}_x1d.fits'.format(**spec.meta).lower()
mast_url = 'https://mast.stsci.edu/api/v0.1/Download/file?uri=mast:JWST/product/' + mast_file

mast = utils.read_catalog(mast_url)

In [ ]:
mast.info()

# Plot the spectrum

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10,5))

pl = ax.plot(spec['wave'], spec['flux'], label='msaexp', color='steelblue', alpha=0.5)
ax.plot(spec['wave'], spec['err'], color=pl[0].get_color(), alpha=0.5)

pl = ax.plot(mast['WAVELENGTH'], mast['FLUX'].to('microJansky'), label='MAST', color='0.3', alpha=0.5)
ax.plot(mast['WAVELENGTH'], mast['FLUX_ERROR'].to('microJansky'), color=pl[0].get_color(), alpha=0.5)

ax.legend()

ax.set_xlabel('wavelength, um')
ax.set_ylabel(r'$f_\nu$ $\mu$Jy')
ax.grid()

# Fitting templates to MSAEXP spectra

In [ ]:
fig, fit_table, fit_result = msaexp.spectrum.fit_redshift(
    spec_file,
    z0=[1,6],
    scale_uncertainty_kwargs={},
)

## Fitting pieces

In [ ]:
# Spectrum object - read the spectrum table and add helper methods for fitting
sampler = msaexp.spectrum.SpectrumSampler(spec_file)

In [ ]:
# Generate a single emission line

line_wave = 3.0 # microns

line_i = sampler.fast_emission_line(
    line_wave,
    line_flux=1.0,
    scale_disp=1.0,
    velocity_sigma=100,
)

fig, axes = plt.subplots(2,1,figsize=(8,5), sharex=True)
for ax in axes:
    ax.plot(sampler['wave'], line_i, color='0.5', alpha=0.5)

# Vary line velocity
ax = axes[0]
for velocity_sigma in [0, 500, 1000, 2000]:
    line_i = sampler.fast_emission_line(
        line_wave,
        line_flux=1.0,
        scale_disp=1.0,
        velocity_sigma=velocity_sigma,
    )
    axes[0].plot(sampler['wave'], line_i, label=f'{velocity_sigma} km/s', alpha=0.5)

leg = ax.legend()
leg.set_title('velocity_sigma')

# Vary scale_disp
ax = axes[1]
for scale_disp in [0.8, 1.3, 1.8]:
    line_i = sampler.fast_emission_line(
        line_wave,
        line_flux=1.0,
        scale_disp=scale_disp,
        velocity_sigma=100,
    )
    axes[1].plot(sampler['wave'], line_i, label=f'R * {scale_disp:.1f}', alpha=0.5)

leg = ax.legend()
leg.set_title('scale_disp')

for ax in axes:
    ax.grid()
    ax.set_xlim(line_wave - 0.1, line_wave + 0.1)

ax.set_xlabel('wavelength, um')
fig.tight_layout(pad=1)


In [ ]:
# Spline functions for continuum
nspline = 21
bsplines = sampler.bspline_array(nspline=nspline, get_matrix=True)

In [ ]:
# Helper for generating emission lines
z = fit_result['z']
_ = msaexp.spectrum.make_templates(sampler, z, bspl=bsplines)
templ_names, is_em_line, templ_matrix = _

print('\n'.join(templ_names))

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(8,5), sharex=True)
_ = axes[0].plot(sampler['wave'], templ_matrix[~is_em_line].T, alpha=0.3, color='tomato')
axes[0].set_ylabel('bsplines')

_ = axes[1].plot(sampler['wave'], templ_matrix[is_em_line].T, alpha=0.3, color='olive')
axes[1].set_ylabel('em lines')
for ax in axes:
    ax.grid()

ax.set_xlabel('wavelength, um')
fig.tight_layout(pad=1)

### Fit the template matrix to the spectrum with least squares


In [ ]:
# Weighted by uncertainties
A = (templ_matrix / sampler['full_err']).T
b = sampler['flux'] / sampler['full_err']

coeffs = np.linalg.lstsq(A[sampler['valid'],:], b[sampler['valid']], rcond=None)

model = templ_matrix.T.dot(coeffs[0])
model_components = templ_matrix.T*coeffs[0]

# Mask for plot
model_components[model_components < 1.e-3] = np.nan

In [ ]:
fig, axes = plt.subplots(2,1,figsize=(8,5))

for ax in axes:
    _ = ax.errorbar(sampler['wave'], sampler['flux'], sampler['full_err'],
                    linestyle='None', alpha=0.3, color='0.3', marker='None')
    
    ax.step(sampler['wave'], sampler['flux'], color='0.2', where='mid', alpha=0.5, label='data')
    
    ax.step(sampler['wave'], model, color='tomato', where='mid', alpha=0.5, label=f'model, z={z:.3f}')
    ax.step(sampler['wave'], model_components, color='tomato', where='mid', alpha=0.2)
    
    ax.grid()

axes[0].set_xlim(1.9, 3.1)

ax.legend()
ax.set_xlabel('wavelength, um')
fig.tight_layout(pad=1)
